In [39]:
!python3 -m venv env
!source env/bin/activate
!python3 -m pip install pandas numpy openpyxl


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [40]:
import pandas as pd
import numpy as np

In [41]:
cities = {'Russia' : ['Moscow', 'Ekaterinburg', 'St.Petersburg'],
          'USA' : ['New York', 'Boston', 'Chicago', 'San Francisco'],
          'Portugal' : ['Lisbon', 'Porto', 'Braga'],
          'Germany' : ['Berlin', 'Hamburg', 'Munich', 'Frankfurt'],
          'France' : ['Paris', 'Marseille', 'Lyon'],
          'Spain' : ['Barcelona', 'Madrid', 'Vigo'],
          'Italy' : ['Rome']}


streets = {'Russia' : ['Lenin Street', '1905 Street', 'Mira Avenue', 'Svobody Street'],
           'USA' : ['Martin Luther King Drive', 'Ross Clark Circle', 'Wall Street', 'Washington Street'],
           'Portugal' : ['Avenida da Liberdade', 'Rua Augusta', 'Rua Cor de Rosa', 'Rua da Bica'],
           'Germany' : ['Bernauer Straße', 'Ebertstraße', 'Chausseestraße', 'Kurfürstendamm'],
           'France' : ['Rue des Barres', 'Rue des Rosiers', 'Rue Chanoinesse', 'Rue Cremieux'],
           'Spain' : ['Calle Huertas', 'Calle Mayor', 'Calle de Preciados', 'Paseo del Prado'],
           'Italy' : ["Campo de' Fiori", "Via della Conciliazione", "Via del Corso", "Via Margutta"]}

zipcodes = {'Moscow' : '125130',
            'Ekaterinburg' : '239032',
            'St.Petersburg': '635342',
            'New York' : '35004',
            'Boston' : '37806',
            'Chicago' : '56004',
            'San Francisco' : '14824',
            'Lisbon' : '145-616',
            'Porto' : '432-516',
            'Braga' : '421-361',
            'Berlin' : '76234',
            'Hamburg' : '25138',
            'Munich' : '27391',
            'Frankfurt' : '56234',
            'Paris' : '57234',
            'Marseille' : '32983',
            'Lyon' : '93274',
            'Barcelona' : '01004',
            'Madrid' : '06923',
            'Vigo' : '33724',
            'Rome' : '98168'
           }

m_names = ['Andrew', 'Vladislav', 'Pedro', 'Jhonny', 
           'David', 'Ricardo', 'Cristiano', 'Jack', 
           'Robert', 'Martin', 'Diego', 'Daniel',
           'Liam', 'Oliver', 'James', 'William']

f_names = ['Ttyana', 'Maria', 'Patricia', 'Ines', 
           'Marianna', 'Anna', 'Julia', 'Helen', 
           'Barbara', 'Babette', 'Jane', 'Olivia',
           'Emma', 'Charlotte', 'Sophia']

l_names = ['Castro', 'Abramov', 'Zurabov', 'Pereira', 'Silva', 
           'Santos', 'Oliveira', 'Motta', 'Smith', 'Williams', 
           'Brown', 'Lavigne', 'Monet', 'Blanchet', 
           'Müller', 'Schneider', 'Schmidt']

categories = {1 : 'Phones',
              2 : 'Wearables',
              3 : 'Home & Care',
              4 : 'Computers',
              5 : 'Tablets',
              6 : 'Audio',
              7 : 'Acessories'}
              
products_file = 'data/products.xlsx'

In [42]:
def clients_data_generation():
    def login_generator(name, last_name, year):
        include_year = np.random.choice([True, False])
        cut_year = np.random.choice([True, False])
        num_of_letters = np.random.randint(2, 6)
        login = name[:num_of_letters-1] + last_name[:num_of_letters]
        if include_year:
            if cut_year:
                login += year[2:]
            else:
                login += year
        return login.lower()

    def email_generator(name, last_name, year):
        nickname = login_generator(name, last_name, str(year)) + '@examplemail.lol'
        return nickname

    my_m_names = np.random.choice(m_names, 1000)
    my_ml_names = np.random.choice(l_names, 1000)

    my_f_names = np.random.choice(f_names, 1000)
    my_fl_names = np.random.choice(l_names, 1000)

    male_df = pd.DataFrame(data = {'First_Name' : my_m_names, 
                                   'Last_Name' : my_ml_names, 
                                   'Sex' : 'M'}).drop_duplicates()
    
    female_df = pd.DataFrame(data = {'First_Name' : my_f_names, 
                                     'Last_Name' : my_fl_names, 
                                     'Sex' : 'F'}).drop_duplicates()
    clients = pd.concat([male_df, female_df])

    clients['Birth_year'] = np.random.randint(1941, 2015, len(clients))
    clients['Phone_number'] = np.random.randint(3*10e+9, 10e+10, len(clients))
    clients['email'] = clients.apply(lambda x: email_generator(x['First_Name'], 
                                                               x['Last_Name'],
                                                               x['Birth_year']), axis = 1)
    clients = clients.sample(frac = 1).reset_index(drop = True)
    clients['ID'] = np.arange(1, len(clients)+1)
    clients = clients[['ID', 'First_Name', 'Last_Name', 'Sex', 'Birth_year', 'Phone_number', 'email']]
    return clients

def address_data_generation(clients):
    def give_addr_2_all_customers(clients):
        countries = np.random.choice(list(cities.keys()), len(clients))
        cur_cities, cur_streets, cur_zipcodes = list(), list(), list()

        for country in countries:
            cur_cities.append(np.random.choice(cities[country]))
            cur_streets.append(np.random.choice(streets[country]))

        for city in cur_cities:
            cur_zipcodes.append(zipcodes[city])

        df = pd.DataFrame(data = {'Country' : countries,
                                  'City' : cur_cities,
                                  'Street' : cur_streets,
                                  'PostCode' : cur_zipcodes,
                                  'CUSTOMER_ID' : clients['ID']})
        return df
    df = give_addr_2_all_customers(clients)
    some_dublicates_df = give_addr_2_all_customers(clients.sample(frac=0.3))
    df = pd.concat([df, some_dublicates_df]).reset_index(drop=True)
    df['Building'] = np.random.randint(1, 200, len(df))
    df['Appartment_tmp'] = np.random.randint(1, 1000, len(df))
    df['Appartment_presence'] = np.random.choice([1,None], p=[0.8, 0.2], size=len(df))
    df['Appartment'] = df['Appartment_tmp'] * df['Appartment_presence']
    df = df.sample(frac=1).reset_index(drop=True)
    df['ID'] = np.arange(1, len(df)+1)
    df = df[['ID', 'Country', 'City', 'Street', 'Building', 'Appartment', 'PostCode', 'CUSTOMER_ID']]
    return df

def products_data_generation(products_file):
    df = pd.read_excel(products_file)
    df = df.drop(columns = ['VAT_ID'])
    return df

In [43]:
clients = clients_data_generation()
addresses = address_data_generation(clients)
products = products_data_generation(products_file)




In [44]:
from datetime import date, timedelta

sdate = date(2020,1,22)   
edate = date(2022,11,22)

dates = pd.date_range(sdate,edate-timedelta(days=1),freq='d').astype(str).to_list()
sample_clients =  np.random.choice(clients['ID'], size=100)

client_dates, client_ids = list(), list()
for client in sample_clients:
    n_orders = np.random.choice([1,2,3,4], p=[0.5, 0.3, 0.1, 0.1])
    curr_dates = list(np.random.choice(dates, n_orders))
    client_dates += curr_dates
    client_ids += [client for _ in curr_dates]

res = pd.DataFrame(data = {'Date' : client_dates, 'CLIENT_ID' : client_ids})
res['Deadline'] = pd.to_datetime(res['Date']) + pd.DateOffset(days=3)
res.sort_values(by = ['Date'], inplace = True)
res['ID'] = np.arange(1, len(res)+1)
res = res[['ID', 'Date', 'Deadline', 'CLIENT_ID']]
res


,ID,Date,Deadline,CLIENT_ID
74,1,2020-01-23,2020-01-26,9
142,2,2020-02-04,2020-02-07,81
108,3,2020-02-04,2020-02-07,422
157,4,2020-02-09,2020-02-12,317
36,5,2020-02-09,2020-02-12,37
...,...,...,...,...
8,179,2022-10-19,2022-10-22,379
153,180,2022-10-22,2022-10-25,268
64,181,2022-10-23,2022-10-26,402
84,182,2022-11-07,2022-11-10,343


In [45]:
products

,ID,SKU_NAME,PRICE,STOCK,CATEGORY_ID,DESCRIPTION
0,1,iPhone 14 PRO,1479.00,13,1,"256GB - Purple - 6,1' - IOS 16"
1,2,iPhone 14,799.00,22,1,"128GB - Black - 6,1' - IOS 16"
2,3,iPhone 14,899.00,23,1,"256GB - Black - 6,1' - IOS 16"
3,4,iPhone 13,699.00,45,1,"256GB - Purple - 6,1' - IOS 16"
4,5,iPhone 13,699.00,12,1,"256GB - Black - 6,1' - IOS 16"
5,6,iPhone 14 Plus,999.00,17,1,"256GB - Black - 6,7' - IOS 17"
6,7,iPhone 14 Plus,899.00,13,1,"128GB - Dark Green - 6,7' - IOS 17"
7,8,iPhone 14 Plus,999.00,15,1,"256GB - Dark Green - 6,7' - IOS 17"
8,9,iPhone 14 PRO max,1629.00,12,1,"512GB - Midnight - 6,7' - IOS 16"
9,10,Samsung Galaxy S22 ULTRA,949.00,11,1,256GB - Black - 6.8' - 12GB


In [46]:
addresses

,ID,Country,City,Street,Building,Appartment,PostCode,CUSTMER_ID
0,1,France,Lyon,Rue Cremieux,188,262,93274,40
1,2,Spain,Madrid,Calle Huertas,67,419,06923,141
2,3,Italy,Rome,Campo de' Fiori,44,919,98168,488
3,4,Russia,Ekaterinburg,Mira Avenue,59,186,239032,119
4,5,USA,San Francisco,Wall Street,114,9,14824,10
...,...,...,...,...,...,...,...,...
663,664,Russia,Ekaterinburg,Lenin Street,101,173,239032,333
664,665,Spain,Vigo,Calle de Preciados,104,322,33724,416
665,666,France,Marseille,Rue des Rosiers,59,460,32983,72
666,667,Spain,Vigo,Calle de Preciados,186,815,33724,233


In [47]:
clients

,ID,First_Name,Last_Name,Sex,Birth_year,Phone_number,email
0,1,Pedro,Blanchet,M,1968,77713067190,pedblan@examplemail.lol
1,2,Barbara,Brown,F,1958,94430519670,bbr1958@examplemail.lol
2,3,Oliver,Zurabov,M,1947,94619737384,olzur47@examplemail.lol
3,4,Barbara,Schmidt,F,1975,95714397493,bsc1975@examplemail.lol
4,5,Patricia,Motta,F,1984,87952430313,patmott@examplemail.lol
...,...,...,...,...,...,...,...
509,510,Jhonny,Zurabov,M,1967,94832175678,jzu@examplemail.lol
510,511,Liam,Smith,M,1988,98891260206,lsm88@examplemail.lol
511,512,Maria,Pereira,F,1946,35136859069,maper@examplemail.lol
512,513,Marianna,Williams,F,1987,56748874299,mwi@examplemail.lol
